In [23]:
import numpy as np
import pandas as pd
import cv2
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [24]:
# 1. Load Dataset
column_names = ['Image', 'Attention']
df = pd.read_csv('../attention.csv', header=None, names=column_names)
df['Image'] = '../resized_images/' + df['Image']  # Tambahkan path folder ke nama gambar

In [25]:
# 2. Ekstraksi Fitur dari Gambar
def extract_features(image_path, resize_dim=(128, 128)):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Gambar tidak ditemukan: {image_path}")
    img = cv2.resize(img, resize_dim)
    
    # Ekstrak HOG Features
    hog_features = hog(
        img, 
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        channel_axis=-1
    )
    
    # Ekstrak Color Histogram (RGB)
    color_hist = cv2.calcHist(
        [img], 
        [0, 1, 2], 
        None, 
        [8, 8, 8], 
        [0, 256, 0, 256, 0, 256]
    ).flatten()
    
    return np.concatenate([hog_features, color_hist])

# Ekstrak fitur untuk semua gambar
X_features = []
valid_images = []
for img_path in df['Image']:
    try:
        features = extract_features(img_path)
        X_features.append(features)
        valid_images.append(True)
    except Exception as e:
        print(f"Error memproses {img_path}: {str(e)}")
        valid_images.append(False)

# Filter data yang valid
df_valid = df[valid_images]
X_features = np.array(X_features)
y_labels = df_valid['Attention'].values



In [26]:
# 3. Preprocessing Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, 
    y_labels, 
    test_size=0.2, 
    stratify=y_labels,
    random_state=42
)

In [27]:
# 4. Bangun Model Neural Network
N = X_train.shape[1]  # Dimensi fitur
num_classes = len(np.unique(y_labels))

model = Sequential([
    Dense(256, activation='relu', input_shape=(N,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

d:\UGM\Kuliah\6th Semester\Pengolahan Citra dan Visi Komputer\PascaUTS\Tugas\Tugas 1\comvis\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
# 5. Pelatihan Model
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=5)],
    verbose=1
)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.2038 - loss: 1.9164 - val_accuracy: 0.2500 - val_loss: 2.0952
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4085 - loss: 1.5265 - val_accuracy: 0.3333 - val_loss: 1.9158
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4500 - loss: 1.4596 - val_accuracy: 0.3333 - val_loss: 1.8096
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3928 - loss: 1.4003 - val_accuracy: 0.3958 - val_loss: 1.7600
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5258 - loss: 1.2023 - val_accuracy: 0.3125 - val_loss: 1.7419
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5374 - loss: 1.2578 - val_accuracy: 0.2917 - val_loss: 1.7636
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5490 - loss: 1.0425 - val_accuracy: 0.3542 - val_loss: 1.7772
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5963 - loss: 1.0368 - val_accuracy: 0.2917 - v

In [29]:
# 6. Evaluasi
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.2500


In [30]:
# Simpan model
model.save('attention_classifier.h5')

# Load model
from tensorflow.keras.models import load_model
loaded_model = load_model('attention_classifier.h5')